In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tqdm

In [2]:
#path = "/Users/user/Desktop/Oka/ML-data/24-1-25"
path ="/Users/aruhy/OneDrive/デスクトップ/data/1-25"

In [3]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','T_cell[℃]','W_ch[mm]','i[A/m^2]'])

T_range = ["/*_100℃_1.5ms","/*_110℃_1.5ms","/*_120℃_1.5ms","/*_130℃_1.5ms","/*_140℃_1.5ms","/*_150℃_1.5ms"]
for T in tqdm.tqdm(T_range):
    W_ch_range = ["_0.4mm","_0.8mm","_1.6mm"] #0.2mmはいったん消す
    for W_ch in W_ch_range:
        files2 = glob.glob(path + T + W_ch)
        for filename in tqdm.tqdm(files2):
            
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'_3D(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            t = re.findall(r'V_(.*?)℃_',filename)
            df1.insert(4,'T_cell[℃]',t[0])
            w = re.findall(r'ms_(.*?)mm',filename)
            df1.insert(5,'W_ch[mm]',w[0])
            df1 = df1.sample(30000)
            df = pd.concat([df, df1])
df['V[V]'] = df['V[V]'].astype(float)
df['T_cell[℃]'] = df['T_cell[℃]'].astype(float)
df['W_ch[mm]'] = df['W_ch[mm]'].astype(float)
df.reset_index(drop=True, inplace=True)
df

  0%|          | 0/11 [00:00<?, ?it/s]

C:\Users\aruhy\AppData\Local\Temp\ipykernel_9772\3462138126.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1])
100%|██████████| 6/6 [03:00<00:00, 30.06s/it]


,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,0.000556,0.005319,0.000550,0.45,100.0,0.4,16860.654509
1,0.000596,0.002167,0.000639,0.45,100.0,0.4,17252.608927
2,0.000068,0.008098,0.000583,0.45,100.0,0.4,17103.468079
3,0.000572,0.000394,0.000593,0.45,100.0,0.4,17296.797515
4,0.000092,0.007486,0.000633,0.45,100.0,0.4,17071.542749
...,...,...,...,...,...,...,...
5909995,0.001890,0.008001,0.000350,0.90,150.0,1.6,1910.361045
5909996,0.001610,0.002955,0.000304,0.90,150.0,1.6,1802.784442
5909997,0.001710,0.000588,0.000349,0.90,150.0,1.6,1980.705700
5909998,0.000610,0.010271,0.000321,0.90,150.0,1.6,1457.001767


In [7]:
def grid_based_sampling(data, grid_count):
    # データの最小値と最大値を計算して、データの範囲を取得
    data_min = np.min(data[:, :3], axis=0)
    data_max = np.max(data[:, :3], axis=0)
    
    # グリッドのサイズを計算
    grid_size = (data_max - data_min) / grid_count
    
   # グリッド内のランダムな代表点を選択してサンプルを構築
    samples = []
    for i in range(grid_count[0]):
        for j in range(grid_count[1]):
            for k in range(grid_count[2]):
                # グリッドの境界を計算
                lower_bound = data_min + np.array([i, j, k]) * grid_size
                upper_bound = lower_bound + grid_size
                
                # グリッド内のデータポイントを抽出
                mask = np.all((data[:, :3] >= lower_bound) & (data[:, :3] < upper_bound), axis=1)
                points_in_grid = data[mask]
                
                if len(points_in_grid) > 0:
                    # グリッド内のデータポイントからランダムに1つ選択して代表点とする
                    selected_point = points_in_grid[np.random.randint(len(points_in_grid))]
                    samples.append(selected_point)
    
    return np.array(samples)

In [6]:
# 各次元ごとのグリッドの数を指定
grid_count = np.array([3000, 3000, 3000])

In [9]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','T_cell[℃]','W_ch[mm]','i[A/m^2]'])

T_range = ["/*_100℃_1.5ms","/*_110℃_1.5ms","/*_120℃_1.5ms","/*_130℃_1.5ms","/*_140℃_1.5ms","/*_150℃_1.5ms"]
for T in tqdm.tqdm(T_range):
    W_ch_range = ["_0.4mm","_0.8mm","_1.6mm"] #0.2mmはいったん消す
    for W_ch in W_ch_range:
        files2 = glob.glob(path + T + W_ch)
        for filename in tqdm.tqdm(files2):
            
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'_3D(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            t = re.findall(r'V_(.*?)℃_',filename)
            df1.insert(4,'T_cell[℃]',t[0])
            w = re.findall(r'ms_(.*?)mm',filename)
            df1.insert(5,'W_ch[mm]',w[0])
            df1 = df1.astype(float)
            df1 = pd.DataFrame(grid_based_sampling(df1.values, grid_count))
            df = pd.concat([df, df1])
df['V[V]'] = df['V[V]'].astype(float)
df['T_cell[℃]'] = df['T_cell[℃]'].astype(float)
df['W_ch[mm]'] = df['W_ch[mm]'].astype(float)
df.reset_index(drop=True, inplace=True)
df

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [58:39<?, ?it/s]


KeyboardInterrupt: 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591000 entries, 0 to 590999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   x座標        591000 non-null  float64
 1   y座標        591000 non-null  float64
 2   z座標        591000 non-null  float64
 3   V[V]       591000 non-null  float64
 4   T_cell[℃]  591000 non-null  float64
 5   W_ch[mm]   591000 non-null  float64
 6   i[A/m^2]   591000 non-null  float64
dtypes: float64(7)
memory usage: 31.6 MB


In [14]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,591000.000000,591000.000000,591000.000000,591000.000000,591000.000000,591000.000000,591000.000000
mean,0.000665,0.005321,0.000434,0.651015,125.025381,0.929949,8391.072422
std,0.000480,0.003107,0.000129,0.157870,17.117832,0.497881,6850.752617
min,0.000004,0.000000,0.000250,0.400000,100.000000,0.400000,-0.086556
25%,0.000290,0.002633,0.000327,0.500000,110.000000,0.400000,2562.444693
50%,0.000580,0.005319,0.000402,0.650000,130.000000,0.800000,6520.303271
75%,0.000930,0.008011,0.000576,0.800000,140.000000,1.600000,12986.673549
max,0.001990,0.010638,0.000650,0.900000,150.000000,1.600000,28991.542841


In [4]:
df_re= df.loc[1770000:1799999,'i[A/m^2]']
df_re

1770000   -0.065324
1770001   -0.037226
1770002   -0.061694
1770003   -0.077847
1770004   -0.014628
             ...   
1799995   -0.051020
1799996   -0.055812
1799997   -0.009965
1799998   -0.023709
1799999   -0.047578
Name: i[A/m^2], Length: 30000, dtype: float64

In [5]:
df.drop(df.loc[1770000:1799999,:].index, inplace=True)

In [15]:
#V=0.65,T=110,W=1.6の値だけ外れていたので補完する
# 条件に基づいて 'i[A/m^2]' のデータを抽出し、その抽出したデータに対して平均を取る
subset1 = df.loc[(df['V[V]'] == 0.6) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']
subset2 = df.loc[(df['V[V]'] == 0.7) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']

# 平均を計算する
#df_re = (subset1.values + subset2.values) / 2
#df.loc[177000:179999,'i[A/m^2]'] = df_re
#df.loc[177000:179999,'x座標'] = df.loc[174000:176999,'x座標']
#df.loc[177000:179999,'y座標'] = df.loc[174000:176999,'y座標']
#df.loc[177000:179999,'z座標'] = df.loc[174000:176999,'z座標']#ランダムサンプリングされているから座標を合わせるのは無理

In [6]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V")

In [7]:
for col in df.columns:
    scaler =  StandardScaler()
    df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1, 1))
    
df.head()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,-0.223982,-0.000244,0.892804,-1.270096,-1.465584,-1.059765,1.232145
1,-0.140361,-1.014150,1.585711,-1.270096,-1.465584,-1.059765,1.289435
2,-1.244160,0.893578,1.150870,-1.270096,-1.465584,-1.059765,1.267636
3,-0.190533,-1.584472,1.226999,-1.270096,-1.465584,-1.059765,1.295894
4,-1.193987,0.696816,1.536679,-1.270096,-1.465584,-1.059765,1.262969


In [17]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,5.880000e+06,5.880000e+06,5.880000e+06,5.880000e+06,5.880000e+06,5.880000e+06,5.880000e+06
mean,-2.170104e-15,-3.403451e-16,1.517835e-15,1.939018e-15,1.530673e-15,-2.881923e-15,6.355636e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.378180e+00,-1.710113e+00,-1.430121e+00,-1.586008e+00,-1.465584e+00,-1.059765e+00,-1.229970e+00
25%,-7.804038e-01,-8.669072e-01,-8.197304e-01,-9.541836e-01,-8.817337e-01,-1.059765e+00,-8.507881e-01
50%,-1.742674e-01,3.719687e-04,-2.416012e-01,-6.447187e-03,2.859677e-01,-2.546722e-01,-2.751684e-01
75%,5.572765e-01,8.654339e-01,1.100484e+00,9.412892e-01,8.698184e-01,1.355513e+00,6.718566e-01
max,2.772809e+00,1.710857e+00,1.666999e+00,1.573114e+00,1.453669e+00,1.355513e+00,3.005577e+00


In [8]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V_std")